## SRO와 RRO의 핵심 차이점

| 구분 | SRO (정적 오버레이) | RRO (동적 오버레이) |
| :--- | :--- | :--- |
| **적용 시점** | 앱 빌드 시점 | 앱 실행 중 (런타임) |
| **적용 방식** | Gradle 빌드 스크립트 설정 | `adb` 명령어 또는 코드로 활성화 |
| **변경 가능성**| 빌드 후 변경 불가능 | 언제든지 활성화/비활성화 가능 |
| **패키지** | 타겟 앱 APK에 포함됨 | 별도의 RRO APK 파일로 존재 |


## SRO 실습 예제 🏗️

###1단계: SROTarget 프로젝트 준비

- 이 앱은 현재 res/values/ 안에 strings.xml과 colors.xml 리소스를 가지고 있습니다.

###2단계: SRO 리소스 폴더 생성
RRO처럼 별도의 안드로이드 모듈을 만드는 것이 아니라, 프로젝트 내에 리소스를 담을 일반 폴더를 생성합니다.

1. Android 스튜디오의 Project 뷰로 전환합니다.

2. 프로젝트의 최상위 폴더(root)에 sro_overlay 라는 새 디렉터리를 만듭니다.

3. sro_overlay 디렉터리 안에 res/values 구조로 폴더를 만듭니다.

4. 새로 만든 sro_overlay/res/values/ 폴더 안에 오버레이할 colors.xml과 strings.xml 파일을 생성합니다.

**프로젝트 구조:**

RROTarget/  
├── app/  
│   ├── src/main/res/  
│   │   └── values/  
│   │       ├── colors.xml  (원본)  
│   │       └── strings.xml (원본)  
│   └── build.gradle.kts  
│
└── sro_overlay/              <-- 새로 만든 SRO 폴더  
    └── res/  
        └── values/  
            ├── colors.xml  (교체할 리소스)  
            └── strings.xml (교체할 리소스)  

sro_overlay/res/values/strings.xml 내용

```xml
<?xml version="1.0" encoding="utf-8"?>
<resources>
    <string name="hello_world">Hello, SRO!</string>
</resources>
```

sro_overlay/res/values/colors.xml 내용

```xml
<?xml version="1.0" encoding="utf-8"?>
<resources>
    <color name="background_color">#4A148C</color>
    <color name="text_color">#F3E5F5</color>
</resources>
```

## 3단계: Gradle 빌드 스크립트 설정 ⚙️
이 단계가 SRO의 핵심입니다. app 모듈의 build.gradle.kts (또는 build.gradle) 파일을 수정하여, 빌드 시 sro_overlay 폴더의 리소스를 추가로 참조하도록 설정합니다.

1. SROTarget 프로젝트의 app/build.gradle.kts 파일을 엽니다.

2. android { ... } 블록 안에 sourceSets 설정을 추가합니다.

build.gradle.kts (Kotlin DSL) 에 추가할 내용:

```Kotlin
android {
    // ... 기존 설정들 ...

    // SRO 설정을 위해 sourceSets 추가
    sourceSets {
        getByName("main") {
            res.srcDirs("src/main/res", "sro_overlay/res")
        }
    }
}
```

파일 수정 후 나타나는 Sync Now 배너를 클릭하여 프로젝트를 동기화합니다.

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/main"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".MainActivity">

    <TextView
        android:id="@+id/hello_text"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="@string/hello_world"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

```

기존 리소스인 colors.xml과 strings.xml내의 중복되는 이름은 제거하거나 주석 처리 합니다.

app/main/res/values/colors.xml

```xml
<?xml version="1.0" encoding="utf-8"?>
<resources>
<!--    <color name="text_color">#FF000000</color>-->
<!--    <color name="background_color">#FFFFFFFF</color>-->
</resources>
```

app/main/res/values/strings.xml

```xml
<?xml version="1.0" encoding="utf-8"?>
<resources>
    <string name="app_name">SROTarget</string>
<!--    <string name="hello_world">hello_world</string>-->
</resources>
```

## 4단계: 빌드 및 결과 확인 ✅
SRO는 별도의 활성화 과정이 필요 없습니다.

1. 안드로이드 스튜디오에서 RROTarget 앱을 평소처럼 빌드하고 실행합니다. (Shift + F10)

2. 앱이 실행되면 즉시 배경색이 보라색으로, 텍스트가 "Hello, SRO!"로 변경되어 나타나는 것을 확인할 수 있습니다.

이는 sro_overlay 폴더의 리소스가 빌드 과정에서 원본 리소스를 덮어쓰고 최종 APK에 포함되었기 때문입니다. ADB 명령어로 활성화/비활성화할 수 없으며, 이 변경은 영구적입니다.

---

## 더 나은 접근 방식: Product Flavors 사용하기 ✅
SRO처럼 빌드 시점에 앱의 버전을 나누는 가장 올바르고 안정적인 방법은 **Product Flavors(제품 버전)**를 사용하는 것입니다. Product Flavors는 main 소스 세트 위에 계층적으로 리소스를 추가하여, 충돌 없이 안전하게 리소스를 덮어쓸 수 있습니다.

###1. build.gradle.kts 설정

android 블록 안에 productFlavors를 설정합니다.

```Kotlin
android {
    ...
    defaultConfig { ... }

    // 1. 버전의 종류를 정의합니다.
    flavorDimensions += "version"

    // 2. 각 버전(flavor)을 만듭니다.
    productFlavors {
        create("original") {
            dimension = "version"
        }
        create("themed") {
            dimension = "version"
        }
    }
}
```

###2. 새로운 폴더 구조
sro_overlay 대신 app/src 폴더 안에 flavor의 이름과 동일한 폴더를 만듭니다.

- 공통 리소스: app/src/main/res/

- themed 버전 전용 리소스: app/src/themed/res/

app/  
└── src/  
    ├── main/  
    │   └── res/  (모든 버전의 공통 리소스)  
    └── themed/  
        └── res/  (themed 버전에서 덮어쓸 리소스만 여기에 넣음)  

###3. themed 버전에 리소스 배치
app/src/themed/res/values/ 폴더를 만들고, 덮어쓰고 싶은 colors.xml 파일을 여기에 넣습니다. main 폴더에 있는 colors.xml은 그대로 둡니다.

###4. 버전 선택 및 실행
- build.gradle.kts 파일을 Sync합니다.

- 안드로이드 스튜디오 Build Variants 탭을 엽니다.  
Menu >> Build >> Select Build Variant...

- Active Build Variant를 themedDebug로 선택합니다.

- 앱을 실행하면, main 리소스 위에 themed 리소스가 덮어씌워져서 충돌 오류 없이 결과물이 나옵니다.